<a href="https://colab.research.google.com/github/zakariajaadi/data-science-portofolio/blob/main/multi_llm_agent_workflow_reports_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authored by : Zakaria Jaadi
# Multi-Agent LLM Workflow for End-to-End Report Automation 🤖 🤝 🤖

Imagine a team of AI agents working together seamlessly to create your reports while you sit back and sip your coffee.

That's exactly what this notebook delivers, an advanced multi-agent LLM workflow designed to automate the entire report generation process on any given topic and produce professional-quality reports.

Each agent plays a specialized role:
* **Research Agent** : searches the web and gathers up-to-date information on the topic.
* **Writer Agent** : Takes the gathered insights and crafts a well-structured report.
* **Review Agent** : Evaluates the report’s quality, ensuring it meets high standards, and can prompt the Writer Agent to refine the report if there are gaps in accuracy, clarity, or coherence.

This isn't just automation; it's a teamwork that delivers superior results while saving you countless hours of manual work.


For this workflow, we’ll be using Gemini 2.0 Flash as the underlying LLM, and Tavily to power the web search.

let's dive in 🚀

In [ ]:
!pip install -q -U llama-index-llms-google-genai
!pip install -q -U llama-index-utils-workflow
!pip install -q -U tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.

In [ ]:
from llama_index.llms.google_genai import GoogleGenAI
from tavily import AsyncTavilyClient

from llama_index.core.agent import FunctionCallingAgent as GenericFunctionCallingAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import (
    Context,
    Event ,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)


##  1- Set API keys

In [ ]:
import os
from google.colab import userdata

google_api_key = userdata.get("GOOGLE_API_KEY")
tavily_api_key = userdata.get("TAVILY_API_KEY")

os.environ["GOOGLE_API_KEY"]=google_api_key

## 2- Create LLM

In [ ]:
llm= GoogleGenAI(model="gemini-2.0-flash")

# Test LLM
print(llm.complete("Who is Ibn Battuta ?"))

INFO:httpx:HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC remote call 1 is done.


Ibn Battuta was a **Moroccan Muslim scholar and explorer who is widely regarded as one of the greatest travelers of all time.** He is famous for his extensive travels throughout the medieval world, covering nearly the entirety of the known Islamic world and beyond.

Here's a breakdown of key aspects of his life and legacy:

*   **Lifespan:** Born in Tangier, Morocco, in 1304 and died around 1368 or 1369.
*   **Travels:** He traveled for nearly 30 years, covering an estimated 73,000 miles (117,000 kilometers). His journeys took him through North Africa, the Middle East, Central Asia, Southeast Asia, India, China, and even parts of Europe.
*   **Purpose of Travel:** While his initial intention was to perform the Hajj (pilgrimage to Mecca), he continued traveling for decades, serving as a judge (qadi) in various places, exploring new cultures, and seeking adventure.
*   **The Rihla:** His travels were documented in a book called "A Gift to Those Who Contemplate the Wonders of Cities and t

## 4 - Create tools

In [ ]:
# Tool that searches the web with tavily
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

## 5- Create Agents

In [ ]:
# Convert our web search functions into a tool
search_web_tool = FunctionTool.from_defaults(fn=search_web)

research_agent = GenericFunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that does research by searching the web and then records the results of your research."
)
write_agent = GenericFunctionCallingAgent.from_tools(
    tools=[],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that writes a report based on the results of research by another agent."
)
review_agent = GenericFunctionCallingAgent.from_tools(
    tools=[],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that reviews a report written by a different agent."
)

## 6- Create agent workflow

Define events classes ( Messages / Triggers passed between agents)

In [ ]:
class ResearchEvent(Event):
    prompt: str

class WriteEvent(Event):
    research: str

class ReviewEvent(Event):
    report: str

class ReviewResults(Event):
    review: str

class RewriteEvent(Event):
    review: str

# Our WriteEvent doesn't need research attached any more (cause it's added to the context)
class WriteEvent(Event):
    pass


Define Agent Workflow class

In [ ]:
class MyMultiAgentFlow(Workflow):

    @step
    async def setup(self, ev: StartEvent) -> ResearchEvent:
        self.research_agent = ev.research_agent
        self.write_agent = ev.write_agent
        self.review_agent = ev.review_agent
        return ResearchEvent(prompt=ev.prompt)

    @step
    async def research(self, ctx: Context, ev: ResearchEvent) -> WriteEvent:

        # Store the prompt in the context
        await ctx.set("prompt", ev.prompt)

        # Do Research
        result = self.research_agent.chat(f"""
        Do a detailed research that another agent will use to write a report about this topic: <topic>{ev.prompt}</topic>.
        You must not call the web search tool multiple times at once.
        """)

        # Store the research in the context for multiple uses
        await ctx.set("research", str(result))

        return WriteEvent()

    @step
    async def write(self, ctx: Context, ev: WriteEvent | RewriteEvent ) -> ReviewEvent:

        # Get research result and prompt from context
        research= await ctx.get('research')
        prompt= await ctx.get('prompt')

        # Create prompt
        prompt = f"""
        Write a detailed report based on this research: <research>{research}</research>
        and this topic: <topic>{prompt}</topic> """

        # Detect RewriteEvents and modify the prompt
        if isinstance(ev, RewriteEvent):
            print("Doing a rewrite!")
            prompt += f"""This report has reviewed and the reviewer had this feedback,
            which you should take into account: <review>{ev.review}</review>"""

        # Hand off to write agent
        result = self.write_agent.chat(prompt)

        return ReviewEvent(report=str(result))

    @step
    async def review(self, ctx: Context, ev: ReviewEvent) -> StopEvent | RewriteEvent:

        # Hand off to review agent
        result = self.review_agent.chat(f"Review this report: {ev.report}")

        # get the LLM to self-reflect
        try_again = llm.complete(f"""
        This is a review of a report written by an agent.
        If you think this review is bad enough that the agent should try again, respond with just the word RETRY.
        If the review is good, reply with just the word CONTINUE. Here's the review: <review>{str(result)}</review>""")

        if try_again.text == "RETRY":
            print("Reviewer said try again")
            return RewriteEvent(review=str(result))
        else:
            print("Reviewer thought it was good!")
            return StopEvent(result=ev.report)

In [ ]:
workflow = MyMultiAgentFlow(timeout=30, verbose=True)

## 7- Plot Agent Workflow

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows
from IPython.display import IFrame
import base64

file_name="workflow.html"
draw_all_possible_flows(workflow, filename=file_name)


# Read and encode the HTML
with open(file_name, "rb") as f:
       encoded_html = base64.b64encode(f.read()).decode("utf-8")


# Display workflow
IFrame(src=f"data:text/html;base64,{encoded_html}", width=800, height=600)

workflow.html


## 8- Run Workflow on this topic : `AI regulation across the European Union in 2025`

In [ ]:
prompt="AI regulation across the European Union in 2025"

In [ ]:
import nest_asyncio
from IPython.display import Markdown, display

nest_asyncio.apply()

workflow = MyMultiAgentFlow(timeout=30, verbose=True)

handler = workflow.run(
    prompt=prompt,
    research_agent=research_agent,
    write_agent=write_agent,
    review_agent=review_agent
)

Running step setup
Step setup produced event ResearchEvent
Running step research
Step research produced event WriteEvent
Running step write
Step write produced event ReviewEvent
Running step review
Reviewer thought it was good!
Step review produced event StopEvent


The workflow execution went as expected 🎉

* the workflow started with web research, followed by write, and ended with review.
* The reviewer thought it was good, completing the workflow.
* Each step produced an event that moved the process to the next stage.

# 9 - Visualize the generated report 🚀

In [ ]:
final_result = await handler
print("==== The report ====")
display(Markdown(final_result))

==== The report ====


## AI Regulation Across the European Union in 2025: A Comprehensive Overview

This report provides an overview of the anticipated AI regulatory landscape across the European Union in 2025, focusing on the key legislation, implementation timelines, and core principles shaping the future of AI development and deployment within the EU. The cornerstone of this regulatory framework is the EU AI Act, designed to foster trustworthy AI while safeguarding fundamental rights, health, safety, and democratic values.

**Key Legislation Driving AI Regulation:**

The EU's approach to AI regulation in 2025 is primarily driven by two key pieces of legislation:

*   **The EU AI Act:** This landmark legislation establishes a comprehensive, risk-based framework for regulating AI systems operating within the EU. Its core objective is to promote the development and adoption of trustworthy AI while mitigating potential risks to individuals and society.
*   **Regulation (EU) 2024/1689:** This regulation plays a crucial role in harmonizing the internal market by creating a uniform legal foundation for the development, marketing, and utilization of AI systems. It aims to ensure a level playing field for businesses and promote innovation within the EU.

**Implementation Timeline and Key Dates:**

While the EU AI Act is a comprehensive piece of legislation, its implementation will be phased. Key milestones expected in 2025 include:

*   **Early 2025:** The initial stages of the AI Act's implementation will begin, focusing on foundational elements such as the definition of an AI system, initiatives to promote AI literacy among the population, and the enforcement of prohibitions on specific AI applications deemed to pose unacceptable risks. This early phase sets the stage for the broader implementation of the Act.
*   **March 7, 2025:** Commission Implementing Regulation (EU) 2025/454 will come into effect. This regulation specifically addresses the operational aspects of Regulation (EU) 2024/1689, particularly concerning the establishment of a scientific panel comprised of independent AI experts. This panel will play a vital role in providing guidance and expertise on AI-related matters.
*   **Full Implementation:** While the exact date is not specified, 2025 is expected to be the deadline for the complete enforcement of prohibitions on certain AI systems deemed to pose unacceptable risks. This signifies a critical step in ensuring that AI systems deployed within the EU adhere to the highest ethical and safety standards.

**Core Principles of the AI Act:**

The EU AI Act is built upon several core principles that guide its regulatory approach:

*   **Risk-Based Approach:** The Act employs a risk-based classification system, categorizing AI systems into four levels: unacceptable, high, limited, and minimal risk. The level of regulatory scrutiny and requirements increases proportionally with the assessed risk level. This allows for a targeted and proportionate approach to regulation.
*   **Prohibited AI Practices:** The Act explicitly prohibits certain AI practices considered to pose unacceptable risks to fundamental rights, safety, and democratic values. These prohibitions aim to prevent the deployment of AI systems that could lead to discriminatory outcomes, mass surveillance, or other harmful consequences.
*   **Requirements for High-Risk AI Systems:** AI systems classified as high-risk are subject to stringent requirements related to data quality, transparency, human oversight, and cybersecurity. These requirements are designed to ensure that high-risk AI systems are reliable, safe, and accountable.
*   **Enforcement and Oversight:** The European Parliament has established a dedicated working group to oversee the implementation and enforcement of the AI Act. This working group will play a crucial role in monitoring compliance, addressing emerging challenges, and ensuring that the Act remains effective in achieving its objectives.

**Key Focus Areas:**

The EU's AI regulatory framework in 2025 is particularly focused on the following areas:

*   **Protection of Fundamental Rights:** A primary objective is to safeguard fundamental rights, including democracy, the rule of law, and environmental protection, from the potential adverse effects of AI systems. This reflects the EU's commitment to ensuring that AI is developed and used in a manner that respects human dignity and promotes social well-being.
*   **Innovation and Competitiveness:** The EU aims to foster innovation in the digital sector while simultaneously ensuring responsible AI development and use. The regulatory framework is designed to strike a balance between promoting technological advancement and mitigating potential risks.
*   **AI Literacy and Awareness:** Recognizing the importance of public understanding and engagement, the regulations emphasize the need for AI literacy initiatives. These initiatives aim to empower citizens to make informed decisions about AI and participate in shaping its future.

**Conclusion:**

The AI regulatory landscape in the EU in 2025 will be significantly shaped by the EU AI Act and related regulations. The risk-based approach, coupled with a focus on fundamental rights, innovation, and AI literacy, reflects the EU's commitment to fostering a responsible and trustworthy AI ecosystem. The successful implementation and enforcement of these regulations will be crucial in ensuring that AI benefits society while mitigating potential risks. The establishment of the scientific panel and the European Parliament's working group demonstrate the EU's dedication to ongoing monitoring and adaptation of the regulatory framework to address the evolving challenges and opportunities presented by AI.


# pretty good result 🎉

# Et Voila ! 🤗